In [75]:
import csv
import numpy as np

def return_i_t_gini(clsarray):
    frequency = [0,0,0]
    count = 0
    for cls in clsarray:
        frequency[int(cls)] += 1
        count += 1
    probarray = [0,0,0]
    for i in range(3):
        probarray[i] = frequency[i]/count
    gini = 0
    for prob in probarray:
        gini += prob*(1-prob)
    #print("Frequency is :", frequency)
    #print("Gini is :",gini)
    return gini

def return_del_t(i_t,t,attr,s):
    left = []
    right = []
    for row in t:
        if (row[attr] < s):
            left.append(row)
        else:
            right.append(row)
    if (left ==[] or right ==[]):
        return 0
    left = np.array(left)
    right = np.array(right)
    #print(attr,s)
    i_t_l = return_i_t_gini(left[:,3])
    i_t_r = return_i_t_gini(right[:,3])
    del_t = i_t - ((1/(len(t)))*(len(left)*i_t_l + len(right)*i_t_r))
    return del_t

def get_dec_node(gini_subtree,subtree):
    del_t_max = [0,0,0]
    for attr in range(3):
        for datapoint in range(len(subtree)):
            value = subtree[datapoint][attr]
            del_t = return_del_t(gini_subtree,subtree,attr,value)
            if (del_t > del_t_max[2]):
                del_t_max = [attr,value,del_t]
    return(del_t_max)

def get_class(subtree):
    frequency = [0,0,0]
    for cls in subtree[:,3]:
        frequency[int(cls)] += 1
    frequency = np.array(frequency)
    #print("Frequency at leaf:", frequency)
    return(frequency.argmax(axis=0))
                
def build_dec_tree(subtree,dec_tree,root_index):
    gini_subtree = return_i_t_gini(subtree[:,3])
    dec_node = get_dec_node(gini_subtree,subtree)
    if ((root_index > 3) or (dec_node == [0,0,0])):
        dec_tree[root_index] = [4,get_class(subtree),0]
        #print("Dec Node at ", root_index,":", [4,get_class(subtree),0])
        #print("Dec Tree till now: ", dec_tree)
        return dec_tree
    dec_tree[root_index] = dec_node
    #print("Dec Node: ", root_index,":", dec_node)
    #print("Dec Tree till now: ", dec_tree)
    left = []
    right = []
    for row in subtree:
        if (row[dec_node[0]] < dec_node[1]):
            left.append(row)
        else:
            right.append(row)
    if (left ==[] or right ==[]):
        return dec_tree
    left = np.array(left)
    right = np.array(right)
    dec_tree = build_dec_tree(left,dec_tree,2*root_index+1)
    dec_tree = build_dec_tree(right,dec_tree,2*root_index+2)
    return dec_tree

def classify(testpts,dec_tree):
    count = 0
    classArr = []
    for testpt in testpts:
        print("For test point:", testpt)
        i=0
        while (i<7):
            print(dec_tree[i])
            if (dec_tree[i][0] == 4):            
                classArr.append(dec_tree[i][1]);
                count += 1
                break
            elif (testpt[int(dec_tree[i][0])]<dec_tree[i][1]):
                i = 2*i+1  
                #print("next node at ",i," : ",dec_tree[i])
            else:
                i = 2*i+2  
                #print("next node at ",i," : ",dec_tree[i])
    return classArr
            
    
with open('02_homework_dataset.csv',newline='') as csvfile:
    datareader = csv.reader(csvfile)
    next(datareader,None)
    dataMatrix = []
    for row in datareader:
        dataR = []
        for j in range(4):
            dataR.append(float(row[j]))
        dataMatrix.append(dataR)
        
    dataMatrix = np.array(dataMatrix) 
    
    dec_tree = [[-1.0,0.0,0.0]]
    dec_tree = np.repeat(dec_tree,7,axis=0)
    
    #gini_tree = return_i_t_gini(dataMatrix[:,3])
    del_t_max = [0,0,0]
    
    print(dataMatrix)
    
    dec_tree = build_dec_tree(dataMatrix,dec_tree,0)
    #for attr in range(3):
    #    for datapoint in range(len(dataMatrix)):
    #        value = dataMatrix[datapoint][attr]
    #        del_t = return_del_t(gini_tree,dataMatrix,attr,value)
    #        if (del_t > del_t_max[2]):
    #            del_t_max = [attr,value,del_t]
        
    #del_t = return_del_t(gini_tree,dataMatrix,0,0.4)
    
    #print(del_t)
    print(dec_tree)
    classArr = classify([[4.1,-0.1,2.2],[6.1,0.4,1.3]], dec_tree)
    print("Test Data Points: ",[[4.1,-0.1,2.2],[6.1,0.4,1.3]]) 
    print("classes: ", classArr)
            

[[ 5.5  0.5  4.5  2. ]
 [ 7.4  1.1  3.6  0. ]
 [ 5.9  0.2  3.4  2. ]
 [ 9.9  0.1  0.8  0. ]
 [ 6.9 -0.1  0.6  2. ]
 [ 6.8 -0.3  5.1  2. ]
 [ 4.1  0.3  5.1  1. ]
 [ 1.3 -0.2  1.8  1. ]
 [ 4.5  0.4  2.   0. ]
 [ 0.5  0.   2.3  1. ]
 [ 5.9 -0.1  4.4  0. ]
 [ 9.3 -0.2  3.2  0. ]
 [ 1.   0.1  2.8  1. ]
 [ 0.4  0.1  4.3  1. ]
 [ 2.7 -0.5  4.2  1. ]]
[[ 0.          4.5         0.36148148]
 [ 4.          1.          0.        ]
 [ 0.          7.4         0.19753086]
 [-1.          0.          0.        ]
 [-1.          0.          0.        ]
 [ 4.          2.          0.        ]
 [ 4.          0.          0.        ]]
For test point: [4.1, -0.1, 2.2]
[ 0.          4.5         0.36148148]
[ 4.  1.  0.]
For test point: [6.1, 0.4, 1.3]
[ 0.          4.5         0.36148148]
[ 0.          7.4         0.19753086]
[ 4.  2.  0.]
Test Data Points:  [[4.1, -0.1, 2.2], [6.1, 0.4, 1.3]]
classes:  [1.0, 2.0]
